# OpenMP Single Line Compute Constructs

In this notebook, we will look at the use of single line directives as a means of moving the computation of a loop to the GPU.
For the examples in this notebook, we will use a simple saxpy code, which we will discuss below.

Let's begin, as ever, by checking that we have suitable GPUs available, moving into the source directory, and making sure that the working environment is clean.

In [ ]:
rocm-smi
cd $HOME/DiRAC-AMD-GPU/notebooks/02-OpenMP/2b-OpenMP_single_line_constructs/C
make clean

## The saxpy code

Saxpy is an acronym that stands for **Single precision A X plus Y**.
It refers to the addition of two vectors, X and Y, where X is multiplied by a scalar A.
Practically, this is done by looping over the elements of X and Y and calculating each result individually.
Mathematically, it can be expressed as;
$$z_i = a \cdot x_i + y_i$$
for the $i^{th}$ element of the arrays.

These sorts of simple vector additions are used extensively in simulation and graphical processing, and can be written into code as a loop over the elements of the two arrays.
Since each calculation has no dependence on any other element in the array, they could, in principle, all be carried out independently.
This makes saxpy a prime candidate for parallelisation across CPUs and GPUs alike.
Indeed, it is often colloquially referred to as the 'Hello, World!' of GPU programming, as the first real code example you are likely to write and run in any new GPU framework.

Let's look now at a traditional CPU implementation of the saxpy code.

## CPU saxpy code

[`saxpy_cpu.c`](./C/saxpy_cpu.c) contains a basic implementation of saxpy code for a CPU.
In the `main` block, we can see the instantiation and initialisation of the two arrays that we will be combining, `x` and `y`.
The `saxpy` function then carries out the vector addition itself within the loop:
```c
for (int i = 0; i < N; i++) {
   y[i] = y[i] + a * x[i];
}
```
Note that in this example, and others that you might see, rather than returning a new vector $z$, we are doing the addition directly into `y`.

You should notice that we already have an OpenMP pragma in this code:
```c
#pragma omp parallel for
```
This instructs the compiler to allow threaded parallelism of the loop on the CPU.

For our example, we initialise the `y` array to `2`, the `x` array to `1` and the scalar `a` to `2`, so we expect every element of the output array `y` after the computation to contain the value `4`.

We can build this code using the `saxpy_cpu` option from the [`Makefile`](./C/Makefile).
Let's build and run it now:

In [ ]:
make saxpy_cpu
./saxpy_cpu

The code has run successfully with the expected output.

Now, let's start changing the pragma instructions to offload our loop onto the GPU.

## Offloading arrays to the GPU

We will now go about moving our calculation to the GPU, starting with a simplified version of the code.

### Automatically and dynamically assigned arrays

The code in [`saxpy_gpu_singleunit_autoalloc.c`](./C/saxpy_gpu_singleunit_autoalloc.c) contains a slightly altered version of the saxpy code, such that vector addition is done not in its own function but in the `main` loop itself.
This allows the compiler more direct information on variables used in the loop, allowing it to better understand when and how it can offload information the the target device. 
This is taken a step further in this code by assigning `x` and `y` on the stack - the compiler is now fully aware of the array sizes that need to be moved in the following loop.
Note that we've had to reduce the number of entries in the arrays in comparison to the previous example, to make sure we don't run out of memory on the stack.

We can see that we've also now changed the OpenMP pragma instruction to:
```c
#pragma omp target teams distribute parallel for
```

Let's remind ourselves of what the directives in this pragma are doing:
 - `target` tells the compiler to transfer control and data from the host (CPU) to the device (GPU),
 - `teams` creates a series of thread teams that can execute the code in parallel,
 - `distribute parallel for` instructs the compiler to run the subsequent `for` loop in parallel, distributing it between the previously created thread teams.

Put all together, this commonly-used construct asks that the subsequent `for` loop block be run in parallel on the GPU.
Let's try building and running this code now, using the `saxpy_gpu_singleunit_autoalloc` option of the [`Makefile`](./C/Makefile).

Remember that we can set the `LIBOMPTARGET_INFO` environment variable to report at runtime the data arguments passed to an OpenMP device kernel.
In this way, we can ensure that we are correctly running on the GPU.

In [ ]:
export LIBOMPTARGET_INFO=1
make saxpy_gpu_singleunit_autoalloc
./saxpy_gpu_singleunit_autoalloc

Success!
We've offloaded our saxpy loop to the GPU, and run the addition there in parallel.


But what if we want to dynamically assign the size of the input arrays, rather than hard-code their sizes at the beginning of the code?
In the example code [`saxpy_gpu_singleunit_dynamic.c`](./C/saxpy_gpu_singleunit_dynamic.c), we've changed their assignment to `malloc` commands whilst keeping the rest of the code the same.
Surely this should run in the same way?


This code can be built using the `saxpy_gpu_singleunit_dynamic` command, so let's build and run it now.

In [ ]:
make saxpy_gpu_singleunit_dynamic
./saxpy_gpu_singleunit_dynamic

Oh no, something's gone wrong!
From the error message, we can see that the GPU is failing to access the data of the array.
In this case, the compiler doesn't have enough information to properly offload the dynamically assigned arrays - without strictly knowing their size, it cannot work out how much memory to assign on the GPU.
Fortunately, for the GPUs we are working on - the MI200 series available on Cosma - we can solve this problem using the `HSA_XNACK` environment variable that we have discussed previously.
Setting this to 1 allows the operating system to automatically assign and move memory between the host and device.

Let's try running that now:

In [ ]:
export HSA_XNACK=1
./saxpy_gpu_singleunit_dynamic

Now we see the expected results.


Of course, attempting to run this code on other GPUs might still have issues - you might need to use a `map` clause in the omp pragma to direct the memory manually.


### Moving saxpy from the main


Now that we can offload our loop successfully, even for dynamically assigned arrays, let's return to the original example code that moved the saxpy operation into a separate function.
[`saxpy_gpu_parallelfor.c`](./C/saxpy_gpu_parallelfor.c) shows how we can apply the new pragma to this code, in a separate function instead of in the `main` block.


Let's compile and run this code, using the `saxpy_gpu_parallelfor` command:

In [ ]:
make saxpy_gpu_parallelfor
./saxpy_gpu_parallelfor

This works as expected.
Note that in this example we still have `HSA_XNACK` set to 1 from the cell in the notebook.
This must be set for this code to run; we will see the same error as before if we were to turn this setting back off.

## The **loop** clause

In more recent versions of OpenMP, the `loop` clause has been added as a simpler replacement to the `distribute parallel for` clause.
The clause that we would need to write for our offloaded loop would then become:
```c
#pragma omp target teams loop
```

This new clause simplifies our usual pragmas, and allows the compiler more freedom in its implementation of parallelism for the applicable loop.
Indeed, ROCm will generate an optimised target region using this pragma over the traditional `distribute parallel for` clause for AMD GPUs.  Additionally, from OpenMP version 6.0 it has changed replace the `teams distribute parallel for` clause. At the time of writing, not many compilers support OpenMP version 6.0, however, you should be aware of this upcoming change and check the documentation for your compiler of choice.

This construct does currently have drawbacks, however.
Because it is relatively novel, not all compilers will support its applications, and the additional freedom that it affords the compilers may not be optimised for all available cards.

There is an example of the `loop` clause for our saxpy code in [`saxpy_gpu_loop.c`](./C/saxpy_gpu_loop.c).  Let's compile and run it now:

In [ ]:
make saxpy_gpu_loop
./saxpy_gpu_loop

We see that the performance is largely consistent with the previous examples.

## The **collapse** clause

The final clause we will look at in this notebook is the `collapse(n)` clause.
This clause requests that `n` nested loops be reduced (or 'collapsed') and executed as a single loop.

We can see an example of this in the example code [`saxpy_gpu_collapse.c`](./C/saxpy_gpu_collapse.c).
Here, we have transformed `x` and `y` into 2D arrays, and are performing the saxpy operation with nested loops over these dimensions.
The `collapse` clause is then added to the end of our normal pragma to request the reduction to one dimension.

In general, this use-case of the `collapse` clause is not recommended for use with C code, as we would usually prefer to transform the 2D array into a single 1D array whose index runs across all elements.
It is, however, very common in Fortran codes that make extensive use of its native multi-dimensional arrays.

Let's build and run the code, using the `saxpy_gpu_collapse` command:

In [ ]:
make saxpy_gpu_collapse
./saxpy_gpu_collapse

As we've seen in this notebook, it is very easy to add a single line pragma that offloads and runs a loop on a GPU.
Dynamic memory options available on the recent MI200 and MI300 series AMD GPUs make managing the memory involved in the process much simpler, so that we can focus on parallelising the appropriate loops rather than fiddling with what memory needs to go where.

In the next section, we will look at some more complex constructs.